In [1]:
import scipy.io
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import math
from sklearn import metrics as met
from sklearn.metrics import accuracy_score
from itertools import *

In [2]:
data = scipy.io.loadmat("C://Users/Home/Downloads/fashion_mnist.mat")
df_X = data['trX']
testX = data['tsX']
dfY = data['trY']
testY = data['tsY']
Y=pd.DataFrame(dfY.T)

In [3]:
#Extract rfeatures by calculating mean and std
def extractFeatures(df):
    X1=[]
    X2=[]
    for i in range(0,len(df)):
        X1.insert(i,np.mean(df[i]))
        X2.insert(i,np.std(df[i]))
    return X1,X2

In [4]:
#Estimation of Mu and Sigma
def parameterEstimation(X):
    m = np.mean(X)
    std = np.std(X)
    dist = stats.norm(m,std)
    return dist

In [5]:
#NaiveClassifier
def naiveClassifier(X,prior_Y0,prior_Y1,dist_X1_0,dist_X2_0,dist_X1_1,dist_X2_1):
    ListY=[]
    for i in range(0,len(X)):
        val0 = dist_X1_0.pdf(X[0][i])*dist_X2_0.pdf(X[1][i])*prior_Y0
        val1 = dist_X1_1.pdf(X[0][i])*dist_X2_1.pdf(X[1][i])*prior_Y1
        ListY.insert(i,0) if (val0>val1) else ListY.insert(i,1)
    return ListY    

In [6]:
#Traindata Feature Extraction - Extract features X1 and X2 and store it in the dataframe df which is the train dataset
df=pd.DataFrame(extractFeatures(df_X))
df=df.T
df.rename(columns={0:'X1',1:'X2'})

,X1,X2
0,0.423159,0.395417
1,0.143367,0.194897
2,0.306057,0.407228
3,0.347949,0.332871
4,0.369848,0.374355
...,...,...
11995,0.168517,0.255543
11996,0.328006,0.362793
11997,0.179372,0.274561
11998,0.189001,0.325558


In [7]:
#TestData Feature Extraction - Extract features X1 and X2 and store it in the dataframe df_testX which is the test dataset
df_testX=pd.DataFrame(extractFeatures(testX))
df_testX=df_testX.T
df_testX.rename(columns={0:'X1',1:'X2'})

,X1,X2
0,0.225665,0.222569
1,0.114816,0.221261
2,0.164591,0.263742
3,0.358593,0.424441
4,0.190886,0.293430
...,...,...
1995,0.096439,0.170104
1996,0.233683,0.383631
1997,0.217972,0.341390
1998,0.175030,0.166916


In [8]:
#Parameter Estimation for Class 0- Estimate Mu,Sigma and for each feature based on the Class 0 and fit them in normal distribution
dist_X1_0=parameterEstimation(df[0][0:6000])
dist_X2_0=parameterEstimation(df[1][0:6000])

In [9]:
#Estimate Mu,Sigma and for each feature based on the Class 1 and fit them in normal distribution
dist_X1_1=parameterEstimation(df[0][6000:12000])
dist_X2_1=parameterEstimation(df[1][6000:12000])

In [10]:
#Estimate prior probabilities of the class Y=0 and Y=1
prior_Y0=len(Y[Y[0]==0.0])/len(Y)
prior_Y1=len(Y[Y[0]==1.0])/len(Y)

In [11]:
#Predict the results for testData and store it in a dataframe
result = pd.DataFrame(naiveClassifier(df_testX,prior_Y0,prior_Y1,dist_X1_0,dist_X2_0,dist_X1_1,dist_X2_1))

In [12]:
result

,0
0,1
1,0
2,1
3,0
4,1
...,...
1995,0
1996,1
1997,1
1998,0


In [13]:
#include the actual testData results in the result dataFrame
flatten_list = list(chain.from_iterable(testY))
len(flatten_list)
result[1]=flatten_list
result

,0,1
0,1,0.0
1,0,1.0
2,1,1.0
3,0,1.0
4,1,1.0
...,...,...
1995,0,1.0
1996,1,1.0
1997,1,1.0
1998,0,0.0


In [14]:
def accuracy_scoreX(X,Y):
    a=0
    for i in range(len(X)):
        if(X[i]==Y[i]):
            a+=1
    return a/len(X)

In [15]:
#Predicting the accuracy of the model
accuracy = accuracy_scoreX(result[1],result[0])
accuracy*=100
print("The accuracy of the model is : "+str(accuracy)+"%")

The accuracy of the model is : 83.15%
